In [ ]:
import time
import torch
from reading_util import filter_unwanted_esm2, H5Dataset
import torch.nn as nn
from FNN_esm import FNN
import numpy as np

In [ ]:

def load_ml_data_emb(path_to_esm2 = "C:/Users/shiko/Downloads/new_esm2_3b.h5", path_to_enzyme_csv = 'C:/Users/shiko/Downloads/new.csv' ):
    """
    Reads in the embeddings and the EC numbers from the h5 file and the csv file and labels them accordingly.
    :param path_to_esm2: path to the h5 file
    :param path_to_enzyme_csv: path to the csv file
    :return: X: embeddings, y: EC numbers (labels)
    """

    to_remove = filter_unwanted_esm2(path_to_enzyme_csv, True)

    h5_dataset = H5Dataset(path_to_esm2, path_to_enzyme_csv)

    loader = torch.utils.data.DataLoader(h5_dataset, batch_size=32, shuffle=False)

    # Iterate over batches
    X = []
    y = []

    t0 = time.time()
    total_count = 0
    for batch in loader:
        total_count += 1
        embs, headers, ec_numbers = batch
        for i in range(len(headers)):
            if headers[i] not in to_remove:
                # here we convert ec to int and do -1
                ec_class = int(ec_numbers[i].split(".")[0]) - 1
                X.append(embs[i].numpy())
                y.append(ec_class)

    # Convert the lists to numpy arrays
    X = np.vstack(X)
    y = np.array(y)

    t1 = time.time()

    total = (t1 - t0) / 60

    print("Total count: ", total_count)
    print(f"\033[32mLOG:\033[0m\n Data loaded in: {round(total, 3)} min")
    print(f"\033[32mLOG:\033[0m\n ESM2 of enzymes: {len(X)}")
    print(f"\033[32mLOG:\033[0m\n Labels of enzymes: {len(y)}")

    return X, y

In [ ]:
model = CNN().to("cuda")
model.load_state_dict(torch.load('FNN_esm.pth'))
model.eval()